# Feedforward Nerual Network ( FNN )

Feedforward is the simplest form of neural network. input data is fed forward from one layer to the next.
Each layer 

## ReLU Activation Function
ReLU is a simple activation function that is used to determine if a neuron should "fire".
It achives this by reciving a numerical value $ z $ this $z$ is in $z \in R$ where $z = w*x+b$.

The z is feed to the ReLU function which $$ReLU(x)= (x)^+ =max(0,x)$$
What this function state is that if a x value is less than zero will be equal to zero. 
Otherwise x = ReLU(x).

## Kuhn Poker
Kuhn poker is an extremely simplified form of poker.
Kuhn as a simple model zero-sum two-player imperfect-information game. 
In Kuhn poker, the deck includes only three playing cards, in this deck there will be a Ace, King and Queen. 

* Play:
    - One card is dealt to each player, which may place bets similarly to a standard poker. Both player have now the option to either bet or pass.
    - If both players bet or both players pass, the player with the higher card wins, otherwise, the betting player wins.

In [1]:
from kuhn import Kuhn

In [4]:
# Setup
kuhn = Kuhn()
kuhn.new_match("human","bot")

# First round / Pre showdown
hands = kuhn.pre_showdown()

# Our hand
print("Our human card", hands["human"])

# Showdown
winner = kuhn.showdown()
print(f"\nHands human: {hands['human']} and bot: {hands['bot']}", "\nWinner is:", winner)

Our human card Q

Hands human: Q and bot: K 
Winner is: bot


In [135]:
def card_compute(card, hand):
    return Kuhn.deck_value[hands]

def expected_outcome(hands, player, opponent):
    prophecy = {('J', 'Q'): ([0, 1], [0, 1]), 
                ('J', 'K'): ([0, 1], [0, 1]), 
                ('Q', 'J'): ([1, 0], [0, 1]),
                ('Q', 'K'): ([1, 0], [1, 0]),
                ('K', 'J'): ([1, 0], [1, 0]),
                ('K', 'Q'): ([1, 0], [1, 0])}
    cards = (hands[player], hands[opponent])
    return prophecy[cards]

def clear_output(output):
    _, index = torch.max(torch.abs(output), dim=0)
    return [1, 0] if index == 0 else [0, 1]

In [80]:
kuhn = Kuhn()
for _ in range(3):
    print(5*"=","New Match", "="*5)
    kuhn.new_match("human", "bot")
    hands = kuhn.pre_showdown()
    print(hands, expected_outcome(hands, "bot", "human"))
    winner = kuhn.showdown()
    print("showdown:", winner)

===== New Match =====
{'human': 'K', 'bot': 'Q'} ([1, 0], [1, 0])
showdown: human
===== New Match =====
{'human': 'J', 'bot': 'Q'} ([1, 0], [0, 1])
showdown: bot
===== New Match =====
{'human': 'J', 'bot': 'Q'} ([1, 0], [0, 1])
showdown: bot


### Neural Networks

In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [43]:
class NeuralNetwork(nn.Module):
    def __init__(self, n_inputs=1, h_layer=3, n_neurons=5, n_output=1):
        super().__init__()

        # Input layer
        self.input_layer = nn.Linear(n_inputs, n_neurons) # Inputs 1 card

        # Hidden layers
        self.hidden_layer = [nn.Linear(n_neurons, n_neurons) for _ in range(h_layer)]

        # Output layer
        self.output_layer = nn.Linear(n_neurons, n_output) # Output: call or fold -> 2 classes call, fold
        self.values = torch.tensor([1.])

    def forward(self, x):
        x = F.relu(self.input_layer(x))

        for layer in self.hidden_layer:
            x = F.relu(layer(x))

        x = self.output_layer(x)
        
        return F.log_softmax(x, dim=0)            

In [179]:
def train(model, epochs):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    kuhn = Kuhn()

    for epoch in range(epochs):
        print(5*"=", f"Epoch {epoch+1}", "="*5)
        # Game new match setup
        kuhn.new_match("human", "bot")
        hands = kuhn.pre_showdown()
        print(f"HANDS: bot: {hands['bot']}, human: {hands['human']}")
        
        model.zero_grad()

        y = torch.tensor( expected_outcome(hands, "bot", "human"), dtype=torch.float)
        y = y.type(torch.LongTensor)

        outcome_all = []
        for i in range(2):
            X = torch.tensor([Kuhn.deck_value[hands["bot"]], i], dtype=torch.float, requires_grad=True)
            output = model(X)
            outcome_all.append(output)
    
            # FOLD
            if clear_output(output) == [0, 1]: break

        if len(outcome_all) == 2:
            winner = kuhn.showdown()
            print(f"Winner: {winner}")
        else:
            print(f'Folded round: {len(outcome_all)}')
        
        print(f"outcome all: {outcome_all}")
        
        for i in range(len(outcome_all)):
                loss = F.nll_loss(outcome_all[i], y[i]) # output = predicted result, y exepected value

                if clear_output(outcome_all[i]) != clear_output(y[i]):
                    torch.neg(loss)

                print("Loss:", loss.item())
                loss.backward()
        optimizer.step()
    return model

In [180]:
EPOCHS = 40
net = NeuralNetwork(n_inputs=2, h_layer=3, n_neurons=20, n_output=2)
net = train(net, EPOCHS)

===== Epoch 1 =====
HANDS: bot: K, human: J
Winner: bot
outcome all: [tensor([-0.8762, -0.5385], grad_fn=<LogSoftmaxBackward0>), tensor([-0.8761, -0.5385], grad_fn=<LogSoftmaxBackward0>)]
Loss: 0.5384934544563293
Loss: 0.5385466814041138
===== Epoch 2 =====
HANDS: bot: Q, human: J
Winner: bot
outcome all: [tensor([-0.8674, -0.5448], grad_fn=<LogSoftmaxBackward0>), tensor([-0.8719, -0.5415], grad_fn=<LogSoftmaxBackward0>)]
Loss: 0.544792890548706
Loss: 0.871906578540802
===== Epoch 3 =====
HANDS: bot: J, human: K
Winner: human
outcome all: [tensor([-0.8659, -0.5459], grad_fn=<LogSoftmaxBackward0>), tensor([-0.8652, -0.5464], grad_fn=<LogSoftmaxBackward0>)]
Loss: 0.865943193435669
Loss: 0.8651662468910217
===== Epoch 4 =====
HANDS: bot: K, human: J
Winner: bot
outcome all: [tensor([-0.8809, -0.5351], grad_fn=<LogSoftmaxBackward0>), tensor([-0.8806, -0.5353], grad_fn=<LogSoftmaxBackward0>)]
Loss: 0.5351054072380066
Loss: 0.5353399515151978
===== Epoch 5 =====
HANDS: bot: K, human: Q
Winne

In [182]:
print(10*"=", "Playing", "="*10)
#NeuralNetwork(n_inputs=2, h_layer=3, n_neurons=7, n_output=2)

kuhn = Kuhn()
matches = 50
w, l, d = 0, 0, 0

for _ in range(matches):
    kuhn.new_match("human", "bot")
    
    hands = kuhn.pre_showdown()
    
    for i in range(2):
        X = torch.tensor([Kuhn.deck_value[hands["bot"]], i], dtype=torch.float)
        output = net(X)
        
        # FOLD
        if clear_output(output) == [0, 1]:
            winner = "fold"
            d += 1
            break

        if i == 1: # -> winner
            winner = kuhn.showdown()
            if winner == "bot":
                w += 1
            else:
                l += 1

    print(f"human: {hands['human']}, bot: {hands['bot']} and bot result", winner)

print(10*"=", "Summary", "="*10)
print('win : ',w)
print('draw: ',d)
print('loss: ',l)

========== Playing ==========
human: K, bot: J and bot result human
human: K, bot: J and bot result human
human: J, bot: K and bot result bot
human: Q, bot: J and bot result human
human: J, bot: Q and bot result bot
human: J, bot: Q and bot result bot
human: J, bot: K and bot result bot
human: K, bot: J and bot result human
human: J, bot: Q and bot result bot
human: K, bot: J and bot result human
human: K, bot: J and bot result human
human: J, bot: Q and bot result bot
human: K, bot: J and bot result human
human: J, bot: Q and bot result bot
human: Q, bot: J and bot result human
human: K, bot: J and bot result human
human: Q, bot: K and bot result bot
human: Q, bot: K and bot result bot
human: K, bot: Q and bot result human
human: Q, bot: J and bot result human
human: K, bot: Q and bot result human
human: K, bot: J and bot result human
human: J, bot: K and bot result bot
human: K, bot: Q and bot result human
human: K, bot: Q and bot result human
human: J, bot: Q and bot result bot
huma